In [1]:
# !pip install pandas
# !pip install nltk kaggle


import os
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\niloa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\niloa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\niloa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
skills = pd.read_csv("C:/Users/niloa/Desktop/MyFiles/SkillsClustering/job_skills.csv")
summary = pd.read_csv(
    "C:/Users/niloa/Desktop/MyFiles/SkillsClustering/job_summary.csv")
title = pd.read_csv("C:/Users/niloa/Desktop/MyFiles/SkillsClustering/linkedin_job_postings.csv", usecols=['job_link', 'job_title'])

#merge all the datsets
firstdf = pd.merge(skills, summary, on='job_link')
df = pd.merge(firstdf, title, on='job_link')
df

,job_link,job_skills,job_summary,job_title
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Housekeeper I - PT
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Assistant General Manager - Huntington 4131
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,School-based Behavior Analyst
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Electrical Deputy Engineering Group Supervisor
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Electrical Assembly Lead
...,...,...,...,...
1296376,https://www.linkedin.com/jobs/view/community-a...,"Communication Skills, Time Management, Custome...",Job Description\nThe Community Ambassador is a...,Community Ambassador - The Station at Raleigh
1296377,https://www.linkedin.com/jobs/view/sr-it-analy...,"Windows SQL, EDI X12, Edifecs Platform, Health...",Interested in fully remote opportunity as a Sr...,Sr. IT Analyst - Edifecs Technical Systems Ana...
1296378,https://www.linkedin.com/jobs/view/operations-...,"Adaptability, Communication, Digital Fluency, ...",Entity:\nProduction & Operations\nJob Family G...,Operations Excellence Specialist
1296379,https://www.linkedin.com/jobs/view/float-patie...,"CNA, EMT, BLS, Medical Assistant, CPCT, LPN, R...","Overview To be part of our organization, every...",Float Patient Care Associate CSO YNHH


In [3]:
# Function to clean and preprcess the skills data

# Ensure necessary NLTK resources are downloaded
try:
    nltk_stopwords = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    nltk_stopwords = set(stopwords.words('english'))

try:
    lemmatizer = WordNetLemmatizer()
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    lemmatizer = WordNetLemmatizer()

# Add additional stopwords
additional_stopwords = {"''", "...", "``", "--"}
stop_words = set(nltk_stopwords)
stop_words.update(additional_stopwords)

# synonym mapping
synonym_map = {
    "machine learning": "ml",
    "artificial intelligence": "ai",
    "customer service": "customer support",
    "javascript": "js",
    "natural language processing": "nlp"
}

def preprocess_skills(skills):
    if skills is None:
        return []

    # Ensure skills is string
    skills_str = str(skills) if not isinstance(skills, str) else skills

    # Split by comma and "and"
    skill_list = []
    for skill_item in skills_str.split(','):
        skill_list.extend([part.strip() for part in re.split(r'\band\b', skill_item) if part.strip()])

    cleaned_skills = []
    punctuation = string.punctuation + "–"

    for skill in skill_list:
        # Apply synonym mapping
        skill_lower = skill.lower()
        for key, value in synonym_map.items():
            skill_lower = skill_lower.replace(key, value)

        # Remove punctuation, non-letters, extra spaces
        skill_lower = ''.join([char for char in skill_lower if char not in punctuation])
        skill_lower = re.sub(r'[^a-z\s]', '', skill_lower)
        skill_lower = re.sub(r'\s+', ' ', skill_lower).strip()

        # Tokenize
        tokens = skill_lower.split()

        # Lemmatize and filter stopwords, short words, and generic terms
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        tokens = [
            word for word in tokens
            if word not in stop_words
            and re.match("^[a-zA-Z]+$", word)
            and len(word) > 3
            and word not in {"ability", "skills", "skill"}
        ]

        cleaned_skills.extend(tokens)

    return cleaned_skills


df['cleaned_skills'] = df['job_skills'].apply(preprocess_skills)

df.head()

,job_link,job_skills,job_summary,job_title,cleaned_skills
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Housekeeper I - PT,"[building, custodial, service, cleaning, janit..."
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Assistant General Manager - Huntington 4131,"[customer, support, restaurant, management, fo..."
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,School-based Behavior Analyst,"[applied, behavior, analysis, data, analysis, ..."
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Electrical Deputy Engineering Group Supervisor,"[electrical, engineering, project, control, sc..."
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Electrical Assembly Lead,"[electrical, assembly, point, point, wiring, s..."


In [4]:
df.to_csv("C:/Users/niloa/Desktop/MyFiles/SkillsClustering/cleaned_skills_df.csv", index=False)